# PaBiRoboy dynamic equations

First, import the necessary functions from SymPy that will allow us to construct time varying vectors in the reference frames.

In [1]:
from __future__ import print_function, division
from sympy import symbols, simplify
from sympy import trigsimp
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point, inertia, RigidBody, KanesMethod
from numpy import deg2rad, rad2deg, array, zeros, linspace

SymPy has a rich printing system. Here we initialize printing so that all of the mathematical equations are rendered in standard mathematical notation.

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

## Interial frames

In [3]:
inertial_frame = ReferenceFrame('I')
lower_leg_left_frame = ReferenceFrame('R_1')
upper_leg_left_frame = ReferenceFrame('R_2')
hip_frame = ReferenceFrame('R_3')
upper_leg_right_frame = ReferenceFrame('R_4')
lower_leg_right_frame = ReferenceFrame('R_5')

## Angles

In [4]:
theta0, theta1, theta2, theta3, theta4, theta5 = dynamicsymbols('theta0, theta1, theta2, theta3, theta4, theta5')

In [5]:
lower_leg_left_frame.orient(inertial_frame, 'Axis', (theta0, inertial_frame.z))
lower_leg_left_frame.dcm(inertial_frame)

Matrix([
[ cos(theta0), sin(theta0), 0],
[-sin(theta0), cos(theta0), 0],
[           0,           0, 1]])

In [6]:
upper_leg_left_frame.orient(lower_leg_left_frame, 'Axis', (theta1, lower_leg_left_frame.z))
simplify(upper_leg_left_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1), sin(theta0 + theta1), 0],
[-sin(theta0 + theta1), cos(theta0 + theta1), 0],
[                    0,                    0, 1]])

In [7]:
hip_frame.orient(upper_leg_left_frame, 'Axis', (theta2, upper_leg_left_frame.z))
simplify(hip_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1 + theta2), sin(theta0 + theta1 + theta2), 0],
[-sin(theta0 + theta1 + theta2), cos(theta0 + theta1 + theta2), 0],
[                             0,                             0, 1]])

In [8]:
upper_leg_right_frame.orient(hip_frame, 'Axis', (theta3, hip_frame.z))
simplify(upper_leg_right_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1 + theta2 + theta3), sin(theta0 + theta1 + theta2 + theta3), 0],
[-sin(theta0 + theta1 + theta2 + theta3), cos(theta0 + theta1 + theta2 + theta3), 0],
[                                      0,                                      0, 1]])

In [9]:
lower_leg_right_frame.orient(upper_leg_right_frame, 'Axis', (theta4, upper_leg_right_frame.z))
simplify(lower_leg_right_frame.dcm(inertial_frame))

Matrix([
[ cos(theta0 + theta1 + theta2 + theta3 + theta4), sin(theta0 + theta1 + theta2 + theta3 + theta4), 0],
[-sin(theta0 + theta1 + theta2 + theta3 + theta4), cos(theta0 + theta1 + theta2 + theta3 + theta4), 0],
[                                               0,                                               0, 1]])

## Points and Locations

In [10]:
ankle_left = Point('AnkleLeft')
knee_left = Point('KneeLeft')
hip_left = Point('HipLeft')
hip_right = Point('HipRight')
knee_right = Point('KneeRight')
ankle_right = Point('AnkleRight')

In [11]:
lower_leg_length, upper_leg_length, hip_length = symbols('l1, l2, l3')

In [12]:
knee_left.set_pos(ankle_left, lower_leg_length * lower_leg_left_frame.y)
knee_left.pos_from(ankle_left).express(inertial_frame).simplify()

- l1*sin(theta0)*I.x + l1*cos(theta0)*I.y

In [13]:
hip_left.set_pos(knee_left, upper_leg_length * upper_leg_left_frame.y)
hip_left.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1))*I.y

In [14]:
hip_right.set_pos(hip_left, hip_length * hip_frame.y)
hip_right.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2))*I.y

In [15]:
knee_right.set_pos(hip_right, upper_leg_length * upper_leg_right_frame.y)
knee_right.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3) + l3*cos(theta0 + theta1 + theta2))*I.y

In [16]:
ankle_right.set_pos(knee_right, upper_leg_length * lower_leg_right_frame.y)
ankle_right.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - l2*sin(theta0 + theta1 + theta2 + theta3 + theta4) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3) + l2*cos(theta0 + theta1 + theta2 + theta3 + theta4) + l3*cos(theta0 + theta1 + theta2))*I.y

In [17]:
lower_leg_left_com_length = lower_leg_length/2
upper_leg_left_com_length = upper_leg_length/2
hip_com_length = hip_length/2
upper_leg_right_com_length = upper_leg_length/2
lower_leg_right_com_length = lower_leg_length/2

lower_leg_left_mass_center = Point('L_COMleft')
upper_leg_left_mass_center = Point('U_COMleft')
hip_mass_center = Point('H_COMleft')
upper_leg_right_mass_center = Point('U_COMright')
lower_leg_right_mass_center = Point('L_COMright')

In [18]:
lower_leg_left_mass_center.set_pos(ankle_left, lower_leg_left_com_length * lower_leg_left_frame.y)
lower_leg_left_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

- l1*sin(theta0)/2*I.x + l1*cos(theta0)/2*I.y

In [19]:
upper_leg_left_mass_center.set_pos(knee_left, upper_leg_left_com_length * upper_leg_left_frame.y)
upper_leg_left_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1)/2)*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1)/2)*I.y

In [20]:
hip_mass_center.set_pos(hip_left, hip_com_length * hip_frame.y)
hip_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l3*sin(theta0 + theta1 + theta2)/2)*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l3*cos(theta0 + theta1 + theta2)/2)*I.y

In [21]:
upper_leg_right_mass_center.set_pos(hip_right, upper_leg_right_com_length * upper_leg_right_frame.y)
upper_leg_right_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3)/2 - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3)/2 + l3*cos(theta0 + theta1 + theta2))*I.y

In [22]:
lower_leg_right_mass_center.set_pos(knee_right, lower_leg_right_com_length * lower_leg_right_frame.y)
lower_leg_right_mass_center.pos_from(ankle_left).express(inertial_frame).simplify()

(-l1*sin(theta0 + theta1 + theta2 + theta3 + theta4)/2 - l1*sin(theta0) - l2*sin(theta0 + theta1) - l2*sin(theta0 + theta1 + theta2 + theta3) - l3*sin(theta0 + theta1 + theta2))*I.x + (l1*cos(theta0 + theta1 + theta2 + theta3 + theta4)/2 + l1*cos(theta0) + l2*cos(theta0 + theta1) + l2*cos(theta0 + theta1 + theta2 + theta3) + l3*cos(theta0 + theta1 + theta2))*I.y

## Kinematical Differential Equations

In [23]:
omega0, omega1, omega2, omega3, omega4 = dynamicsymbols('omega0, omega1, omega2, omega3, omega4')
kinematical_differential_equations = [omega0 - theta0.diff(),
                                      omega1 - theta1.diff(),
                                      omega2 - theta2.diff(),
                                      omega3 - theta3.diff(),
                                      omega4 - theta4.diff(),]
kinematical_differential_equations

[omega0 - theta0', omega1 - theta1', omega2 - theta2', omega3 - theta3', omega4 - theta4']

## Angular Velocities

In [24]:
lower_leg_left_frame.set_ang_vel(inertial_frame,         omega0 * inertial_frame.z)
lower_leg_left_frame.ang_vel_in(inertial_frame)

omega0*I.z

In [25]:
upper_leg_left_frame.set_ang_vel(lower_leg_left_frame,   omega1     * inertial_frame.z)
upper_leg_left_frame.ang_vel_in(inertial_frame)

(omega0 + omega1)*I.z

In [26]:
hip_frame.set_ang_vel(upper_leg_left_frame,              omega2     * inertial_frame.z)
hip_frame.ang_vel_in(inertial_frame)

(omega0 + omega1 + omega2)*I.z

In [27]:
upper_leg_right_frame.set_ang_vel(hip_frame,             omega3     * inertial_frame.z)
upper_leg_right_frame.ang_vel_in(inertial_frame)

(omega0 + omega1 + omega2 + omega3)*I.z

In [28]:
lower_leg_right_frame.set_ang_vel(upper_leg_right_frame, omega4     * inertial_frame.z)
lower_leg_right_frame.ang_vel_in(inertial_frame)

(omega0 + omega1 + omega2 + omega3 + omega4)*I.z

# Linear Velocities

Finally, the linear velocities of the mass centers are needed. Starting at the ankle which has a velocity of 0.

In [29]:
ankle_left.set_vel(inertial_frame, 0)

Working our way up the chain we can make use of the fact that the joint points are located on two rigid bodies. Any fixed point in a reference frame can be computed if the linear velocity of another point on that frame is known and the frame's angular velocity is known.

$$^I\mathbf{v}^{P_2} = ^I\mathbf{v}^{P_1} + ^I\omega^A \times \mathbf{r}^{\frac{P_2}{P_1}}$$

The `Point.v2pt_theory()` method makes it easy to do this calculation.

In [30]:
lower_leg_left_mass_center.v2pt_theory(ankle_left, inertial_frame, lower_leg_left_frame)
lower_leg_left_mass_center.vel(inertial_frame)

- l1*omega0/2*R_1.x

In [31]:
knee_left.v2pt_theory(ankle_left, inertial_frame, lower_leg_left_frame)
knee_left.vel(inertial_frame)

- l1*omega0*R_1.x

In [32]:
upper_leg_left_mass_center.v2pt_theory(knee_left, inertial_frame, upper_leg_left_frame)
upper_leg_left_mass_center.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)/2*R_2.x

In [33]:
hip_left.v2pt_theory(knee_left, inertial_frame, upper_leg_left_frame)
hip_left.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)*R_2.x

In [34]:
hip_mass_center.v2pt_theory(hip_left, inertial_frame, hip_frame)
hip_mass_center.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)*R_2.x - l3*(omega0 + omega1 + omega2)/2*R_3.x

In [35]:
hip_right.v2pt_theory(hip_left, inertial_frame, hip_frame)
hip_right.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)*R_2.x - l3*(omega0 + omega1 + omega2)*R_3.x

In [36]:
knee_right.v2pt_theory(hip_right, inertial_frame, upper_leg_right_frame)
knee_right.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)*R_2.x - l3*(omega0 + omega1 + omega2)*R_3.x - l2*(omega0 + omega1 + omega2 + omega3)*R_4.x

In [37]:
upper_leg_right_mass_center.v2pt_theory(knee_right, inertial_frame, upper_leg_right_frame)
upper_leg_right_mass_center.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)*R_2.x - l3*(omega0 + omega1 + omega2)*R_3.x - l2*(omega0 + omega1 + omega2 + omega3)/2*R_4.x

In [38]:
ankle_right.v2pt_theory(knee_right, inertial_frame, lower_leg_right_frame)
ankle_right.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)*R_2.x - l3*(omega0 + omega1 + omega2)*R_3.x - l2*(omega0 + omega1 + omega2 + omega3)*R_4.x - l2*(omega0 + omega1 + omega2 + omega3 + omega4)*R_5.x

In [39]:
lower_leg_right_mass_center.v2pt_theory(knee_right, inertial_frame, lower_leg_right_frame)
lower_leg_right_mass_center.vel(inertial_frame)

- l1*omega0*R_1.x - l2*(omega0 + omega1)*R_2.x - l3*(omega0 + omega1 + omega2)*R_3.x - l2*(omega0 + omega1 + omega2 + omega3)*R_4.x - l1*(omega0 + omega1 + omega2 + omega3 + omega4)/2*R_5.x

# Masses, Inertia, Rigid Bodies

In [40]:
lower_leg_mass, upper_leg_mass, hip_mass = symbols('m_L, m_U, m_H')

In [41]:
lower_leg_inertia, upper_leg_inertia, hip_inertia = symbols('I_Lz, I_Uz, I_Hz')

In [42]:
lower_leg_left_inertia_dyadic = inertia(lower_leg_left_frame, 0, 0, lower_leg_inertia)
lower_leg_left_central_inertia = (lower_leg_left_inertia_dyadic, lower_leg_left_mass_center)
lower_leg_left_inertia_dyadic.to_matrix(lower_leg_left_frame)

Matrix([
[0, 0,    0],
[0, 0,    0],
[0, 0, I_Lz]])

In [43]:
upper_leg_left_inertia_dyadic = inertia(upper_leg_left_frame, 0, 0, upper_leg_inertia)
upper_leg_left_central_inertia = (upper_leg_left_inertia_dyadic, upper_leg_left_mass_center)
upper_leg_left_inertia_dyadic.to_matrix(upper_leg_left_frame)

Matrix([
[0, 0,    0],
[0, 0,    0],
[0, 0, I_Uz]])

In [44]:
hip_inertia_dyadic = inertia(hip_frame, 0, 0, hip_inertia)
hip_central_inertia = (hip_inertia_dyadic, hip_mass_center)
hip_inertia_dyadic.to_matrix(hip_frame)

Matrix([
[0, 0,    0],
[0, 0,    0],
[0, 0, I_Hz]])

In [45]:
upper_leg_right_inertia_dyadic = inertia(upper_leg_right_frame, 0, 0, upper_leg_inertia)
upper_leg_right_central_inertia = (upper_leg_right_inertia_dyadic, upper_leg_right_mass_center)
upper_leg_right_inertia_dyadic.to_matrix(upper_leg_right_frame)

Matrix([
[0, 0,    0],
[0, 0,    0],
[0, 0, I_Uz]])

In [46]:
lower_leg_right_inertia_dyadic = inertia(lower_leg_right_frame, 0, 0, lower_leg_inertia)
lower_leg_right_central_inertia = (lower_leg_right_inertia_dyadic, lower_leg_right_mass_center)
lower_leg_right_inertia_dyadic.to_matrix(lower_leg_right_frame)

Matrix([
[0, 0,    0],
[0, 0,    0],
[0, 0, I_Lz]])

In [47]:
lower_leg_left = RigidBody('Lower Leg Left', lower_leg_left_mass_center, lower_leg_left_frame,
                      lower_leg_mass, lower_leg_left_central_inertia)

In [48]:
upper_leg_left = RigidBody('Upper Leg Left', upper_leg_left_mass_center, upper_leg_left_frame,
                      upper_leg_mass, upper_leg_left_central_inertia)

In [49]:
hip = RigidBody('Hip', hip_mass_center, hip_frame, hip_mass, hip_central_inertia)

In [50]:
upper_leg_right = RigidBody('Upper Leg Right', upper_leg_right_mass_center, upper_leg_right_frame,
                      upper_leg_mass, upper_leg_right_central_inertia)

In [51]:
lower_leg_right = RigidBody('Lower Leg Right', lower_leg_right_mass_center, lower_leg_right_frame,
                      lower_leg_mass, lower_leg_right_central_inertia)

# Forces and Torques

In [52]:
g = symbols('g')

In [53]:
lower_leg_left_grav_force = (lower_leg_left_mass_center, -lower_leg_mass * g * inertial_frame.y)
upper_leg_left_grav_force = (upper_leg_left_mass_center, -upper_leg_mass * g * inertial_frame.y)
hip_grav_force = (hip_mass_center, -hip_mass * g * inertial_frame.y)
upper_leg_right_grav_force = (upper_leg_right_mass_center, -upper_leg_mass * g * inertial_frame.y)
lower_leg_right_grav_force = (lower_leg_right_mass_center, -lower_leg_mass * g * inertial_frame.y)

In [54]:
ankle_left_torque, knee_left_torque, hip_left_torque, hip_right_torque, knee_right_torque, ankle_right_torque = dynamicsymbols('T_a0, T_k0, T_h0, T_h1, T_k1, T_a1')

In [55]:
lower_leg_left_torque_vector = ankle_left_torque * inertial_frame.z - knee_left_torque * inertial_frame.z
upper_leg_left_torque_vector = knee_left_torque * inertial_frame.z - hip_left_torque * inertial_frame.z
hip_left_torque_vector = hip_left_torque * inertial_frame.z - hip_right_torque * inertial_frame.z
hip_right_torque_vector = hip_right_torque * inertial_frame.z - knee_right_torque * inertial_frame.z
upper_leg_right_torque_vector = knee_right_torque * inertial_frame.z - ankle_right_torque * inertial_frame.z
lower_leg_right_torque_vector = ankle_right_torque * inertial_frame.z

In [56]:
lower_leg_left_torque = (lower_leg_left_frame, lower_leg_left_torque_vector)
upper_leg_left_torque = (upper_leg_left_frame, upper_leg_left_torque_vector)
hip_left_torque = (hip_frame, hip_left_torque_vector)
hip_right_torque = (hip_frame, hip_right_torque_vector)
upper_leg_right_torque = (upper_leg_right_frame, upper_leg_right_torque_vector)
lower_leg_right_torque = (lower_leg_right_frame, lower_leg_right_torque_vector)

# Equations of Motion

In [57]:
coordinates = [theta0, theta1, theta2, theta3, theta4]
coordinates

[theta0, theta1, theta2, theta3, theta4]

In [58]:
speeds = [omega0, omega1, omega2, omega3, omega4]
speeds

[omega0, omega1, omega2, omega3, omega4]

In [59]:
kinematical_differential_equations

[omega0 - theta0', omega1 - theta1', omega2 - theta2', omega3 - theta3', omega4 - theta4']

In [60]:
kane = KanesMethod(inertial_frame, coordinates, speeds, kinematical_differential_equations)

In [61]:
loads = [lower_leg_left_grav_force,
         upper_leg_left_grav_force,
         hip_grav_force, 
         upper_leg_right_grav_force,
         lower_leg_right_grav_force,
         lower_leg_left_torque,
         upper_leg_left_torque,
         hip_left_torque,
         hip_right_torque,
         upper_leg_right_torque,
         lower_leg_right_torque]
loads

[(L_COMleft, - g*m_L*I.y),
 (U_COMleft, - g*m_U*I.y),
 (H_COMleft, - g*m_H*I.y),
 (U_COMright, - g*m_U*I.y),
 (L_COMright, - g*m_L*I.y),
 (R_1, (T_a0 - T_k0)*I.z),
 (R_2, (-T_h0 + T_k0)*I.z),
 (R_3, (T_h0 - T_h1)*I.z),
 (R_3, (T_h1 - T_k1)*I.z),
 (R_4, (-T_a1 + T_k1)*I.z),
 (R_5, T_a1*I.z)]

In [62]:
bodies = [lower_leg_left, upper_leg_left, hip, upper_leg_right, lower_leg_right]
bodies

[Lower Leg Left, Upper Leg Left, Hip, Upper Leg Right, Lower Leg Right]

In [63]:
fr, frstar = kane.kanes_equations(loads, bodies)

In [ ]:
trigsimp(fr + frstar)

Matrix([
[g*l1*m_H*sin(theta0) + g*l1*m_L*sin(theta0 + theta1 + theta2 + theta3 + theta4)/2 + 3*g*l1*m_L*sin(theta0)/2 + 2*g*l1*m_U*sin(theta0) + g*l2*m_H*sin(theta0 + theta1) + g*l2*m_L*sin(theta0 + theta1) + g*l2*m_L*sin(theta0 + theta1 + theta2 + theta3) + 3*g*l2*m_U*sin(theta0 + theta1)/2 + g*l2*m_U*sin(theta0 + theta1 + theta2 + theta3)/2 + g*l3*m_H*sin(theta0 + theta1 + theta2)/2 + g*l3*m_L*sin(theta0 + theta1 + theta2) + g*l3*m_U*sin(theta0 + theta1 + theta2) + l1**2*m_L*(omega0 + omega1 + omega2 + omega3 + omega4)**2*sin(theta1 + theta2 + theta3 + theta4)/2 - l1**2*m_L*omega0**2*sin(theta1 + theta2 + theta3 + theta4)/2 + l1*l2*m_H*(omega0 + omega1)**2*sin(theta1) - l1*l2*m_H*omega0**2*sin(theta1) - l1*l2*m_L*(omega0 + omega1)**2*sin(theta2 + theta3 + theta4)/2 + l1*l2*m_L*(omega0 + omega1)**2*sin(theta1) + l1*l2*m_L*(omega0 + omega1 + omega2 + omega3)**2*sin(theta1 + theta2 + theta3) - l1*l2*m_L*(omega0 + omega1 + omega2 + omega3)**2*sin(theta4)/2 + l1*l2*m_L*(omega0 + omega1 +

Keep in mind that out utlimate goal is to have the equations of motion in first order form:

$$ \dot{\mathbf{x}} = \mathbf{g}(\mathbf{x}, t) $$

The equations of motion are linear in terms of the derivatives of the generalized speeds and the `KanesMethod` class automatically puts the equations in a more useful form for the next step of numerical simulation:

$$ \mathbf{M}(\mathbf{x}, t)\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, t) $$

Note that

$$ \mathbf{g} = \mathbf{M}^{-1}(\mathbf{x}, t) \mathbf{f}(\mathbf{x}, t) $$

and that $\mathbf{g}$ can be computed analytically but for non-toy problems, it is best to do this numerically. So we will simply generate the $\mathbf{M}$ and $\mathbf{f}$ matrices for later use.

The mass matrix, $\mathbf{M}$, can be accessed with the `mass_matrix` method (use `mass_matrix_full` to include the kinematical differential equations too. We can use `trigsimp` again to make this relatively compact: 

In [ ]:
mass_matrix = trigsimp(kane.mass_matrix_full)
mass_matrix

The right hand side, $\mathbf{f}$, is a vector function of all the non-inertial forces (gyroscopic, external, coriolis, etc):

In [ ]:
forcing_vector = trigsimp(kane.forcing_full)
forcing_vector

# Simulation

In [ ]:
from scipy.integrate import odeint
from pydy.codegen.ode_function_generators import generate_ode_function
%matplotlib inline
from matplotlib.pyplot import plot, legend, xlabel, ylabel, rcParams
rcParams['figure.figsize'] = (14.0, 6.0)

In [ ]:
constants = [lower_leg_length,
             upper_leg_length,
             hip_length,
             
             lower_leg_left_com_length,
             upper_leg_left_com_length,
             hip_com_length,
             upper_leg_right_com_length,
             lower_leg_right_com_length,
             
             lower_leg_mass,
             upper_leg_mass,
             hip_mass,
             
             lower_leg_inertia,
             upper_leg_inertia,
             hip_inertia,
             
             g]
constants

In [ ]:
numerical_constants = array([0.42,  # lower_leg_length [m]
                             0.54, # upper_leg_length [m]
                             0.2, # hip_length
                             
                             0.21,  # lower_leg_com_length [m]
                             0.27,  # upper_leg_com_length
                             0.10,  # hip_com_length
                             0.27,  # upper_leg_com_length
                             0.21,  # lower_leg_com_length [m]
                             
                             1.0,  # lower_leg_mass [kg]
                             1.5,  # upper_leg_mass [kg]
                             2.0,  # hip_mass [kg]
                             
                             0.1,  # lower_leg_inertia [kg*m^2]                             
                             0.2,  # upper_leg_inertia [kg*m^2]
                             0.1,  # hip_inertia [kg*m^2]
                             
                             9.81],  # acceleration due to gravity [m/s^2]
                            ) 
numerical_constants

In [ ]:
coordinates = [theta0, theta1, theta2, theta3, theta4]
coordinates

In [ ]:
speeds = [omega0, omega1, omega2, omega3, omega4]
speeds

In [ ]:
specified = [ankle_left_torque, knee_left_torque, hip_left_torque, hip_right_torque, knee_right_torque, ankle_right_torque]

In [ ]:
right_hand_side = generate_ode_function(forcing_vector, coordinates,
                                        speeds, constants,
                                        mass_matrix=mass_matrix,
                                        specifieds=specified)

In [ ]:
x0 = zeros(10)
x0

In [ ]:
x0[:6] = deg2rad(2.0)
x0

In [ ]:
numerical_specified = zeros(5)

args = {'constants': numerical_constants,
        'specified': numerical_specified}
frames_per_sec = 60
final_time = 5.0

t = linspace(0.0, final_time, final_time * frames_per_sec)

In [ ]:
right_hand_side(x0, 0.0, numerical_specified, numerical_constants)

In [ ]:
upper_leg_left_torque